# Comparing multivariate analysis using diagonal priors with univariate computations

This is to verify that the mmbr implementation is correct for the truthly multivariate computations. Previously I have only compared it for the degenerated case where the prior matrix is 1 by 1.

## Simulate data

In [1]:
set.seed(1)
n = 1000
p = 1000
X = matrix(rnorm(n*p),nrow=n,ncol=p)
beta1 = beta2 = rep(0,p)
beta1[1:4] = runif(4)
beta2[1:4] = runif(4)
y1 = X %*% beta1 + rnorm(n)
y2 = X %*% beta2 + rnorm(n)

In [2]:
beta1[1:4]

[1] 0.6143060 0.3201991 0.2052207 0.8312957

In [3]:
beta2[1:4]

[1] 0.1468283 0.3689267 0.7327256 0.7734084

## Run univariate computation

In [4]:
m1 = susieR::susie(X,y1,residual_variance = var(y1), estimate_residual_variance=F, scaled_prior_variance = 0.2, L=10)
bhat1 = coef(m1)

In [5]:
m2 = susieR::susie(X,y2,residual_variance = var(y2), estimate_residual_variance=F, scaled_prior_variance = 0.2, L=10)
bhat2 = coef(m2)

In [6]:
bhat1[2:5]

[1] 6.474179e-01 3.626992e-01 1.969718e-11 8.193507e-01

In [7]:
bhat2[2:5]

[1] 3.729464e-13 3.863424e-01 7.434874e-01 7.356046e-01

## Run multivariate computation

In [8]:
prior_var = diag(0.2 * c(var(y1), var(y2)))
residual_var = diag(c(var(y1), var(y2)))
y = cbind(y1,y2)
data = mmbr:::DenseData$new(X,y)

In [9]:
prior_covar = mmbr:::MashInitializer$new(list(prior_var), 1, 1, 0)

In [10]:
m = mmbr::susie(X, y, L=10, V=prior_covar)

[1] 1


In [11]:
length(m$coef)

[1] 1002

## Get lfsr

In [12]:
mmbr::mmbr_get_cs_lfsr(m)

0.000000e+00,0.000000e+00
1.198719e-04,0.000000e+00
0.000000e+00,5.209887e-05
5.884182e-15,0.000000e+00
1.000000e+00,1.000000e+00
1.000000e+00,1.000000e+00
1.000000e+00,1.000000e+00
1.000000e+00,1.000000e+00
1.000000e+00,1.000000e+00
1.000000e+00,1.000000e+00


## Get PIP per condition

In [14]:
pip_cond = mmbr:::mmbr_get_pip_per_condition(m, prior_covar)

In [15]:
dim(pip_cond)

[1] 1000    2

In [16]:
head(pip_cond)

1.000000000,1.000000000
1.000000000,1.000000000
1.000000000,1.000000000
1.000000000,1.000000000
0.004774091,0.004774091
0.015642346,0.015642346
